In [ ]:
from modelzipper.tutils import *

content = auto_read_data("/data/zecheng/sunzc/result/dzc_res/longbench/Llama-3-8B-Instruct-80K-QLoRA/gov_report.json")

In [3]:
from modelzipper.tutils import *
import transformers
import sys
sys.path.append('/data/zecheng/Retrieval_Head/quick_eval')
from utils import all_datasets, longbench_dataset_prompt, longbench_pred_length, load_model, DATASET2CATEGORY
from chat import apply_chat_template

model_path = '/data/zecheng/hf_models/longchat-7b-v1.5-32k'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

test_model = load_model(
    model_name='llama2', model_path=model_path, 
    adapter_path='/nvme/zecheng/ckpt/simpo-llama2_fix/checkpoint-300/context_scaling', 
    max_training_length=16384, max_testing_length=32000, use_logn=True, device=torch.device('cuda:6'),
)

# model = transformers.AutoModelForCausalLM.from_pretrained(
#     model_path, 
#     attn_implementation="flash_attention_2", 
#     torch_dtype=torch.bfloat16, use_cache=True
# ).to('cuda')

dataset_name = 'passage_count'
dir_path = '/data/zecheng/Retrieval_Head/quick_eval/data'
test_data = auto_read_data(os.path.join(dir_path, f"{dataset_name}.jsonl"))

PROMPT_TEMPLATE, PRED_LENGTH = longbench_dataset_prompt[dataset_name], longbench_pred_length[dataset_name]
samples = [test_data[i] for i in range(150, len(test_data))]

for sample in samples:
    context, input_, answers = sample['context'], sample['input'], sample['answers']
    prompt = PROMPT_TEMPLATE.format(input=input_, context=context)
    prompt = apply_chat_template(
        "llama-2", 
        messages=[{'role': 'user', 'content': prompt}],
        tokenizer=tokenizer,
        add_generation_prompt=True,
    ).raw

    textual_input = tokenizer(prompt, return_tensors="pt").to(test_model.device).input_ids

    eos_token_id = tokenizer.eos_token_id
    if isinstance(eos_token_id, int):
        eos_token_id = [eos_token_id]
    eos_token_id.append(tokenizer.encode("\n", add_special_tokens=False)[-1])

    outputs = test_model.generate(
        textual_input, 
        min_new_tokens=1,
        max_new_tokens=PRED_LENGTH, 
        do_sample=None,
        begin_suppress_tokens=eos_token_id,
        eos_token_id=eos_token_id, 
        temperature=None,
        top_p=None,
        )

    pred_str = tokenizer.decode(outputs[0, textual_input.shape[-1]:], skip_special_tokens=True)

    print(f'prediction: {pred_str} | Answer: {answers[0]}')

|||---- Using logn scaling... ----|||


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]
/data/anaconda3/envs/zecheng/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/data/anaconda3/envs/zecheng/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and s

prediction: There are some paragraphs below sourced from Wikipedia Some of them may be duplicates Please carefully read these paragraphs and determine how many non-repeating | Answer: 17
prediction: There are some paragraphs below sourced from Wikipedia Some of them may be duplicates Please carefully read these paragraphs and determine how many unique paragraphs there are | Answer: 27
prediction: There are some paragraphs below sourced from Wikipedia Some of them may be duplicates Please carefully read these paragraphs and determine how many unique paragraphs there are | Answer: 7
prediction: There are some paragraphs below sourced from Wikipedia Some of them may be duplicates Please carefully read these paragraphs and determine how many non-repeating | Answer: 14
prediction: There are some paragraphs below sourced from Wikipedia Some of them may be duplicates Please carefully read these paragraphs and determine how many unique paragraphs there are | Answer: 31
prediction: There are so

In [5]:
answers

['Yanzhou']

In [7]:
prompt.index('Kaiyuan')

161

In [8]:
prompt[140: 200]

'passages.\nPassage 1:\nKaiyuan, Liaoning\nKaiyuan (simplified C'

## evaluation

In [2]:
import os
import json
import argparse
import numpy as np
from fire import Fire
from modelzipper.tutils import *
from utils import *
from metrics import (
    qa_f1_score, rouge_score, classification_score, 
    retrieval_score, count_score, code_sim_score,
)

dataset2metric = {
    "narrativeqa": qa_f1_score,
    "qasper_e": qa_f1_score,
    "multifieldqa_en": qa_f1_score,
    "hotpotqa_e": qa_f1_score,
    "2wikimqa_e": qa_f1_score,
    "musique": qa_f1_score,
    "gov_report_e": rouge_score,
    "multi_news_e": rouge_score,
    "trec": classification_score,
    "triviaqa_e": qa_f1_score,
    "samsum_e": rouge_score,
    "passage_retrieval_en": retrieval_score,
    "passage_count": count_score,
    "lcc_e": code_sim_score,
    "repobench-p_e": code_sim_score,
    "qmsum_e": rouge_score,
}


def parse_args(args=None):
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, default=None)
    parser.add_argument('--e', action='store_true', help="Evaluate on LongBench-E")
    return parser.parse_args(args)

def scorer_e(dataset, predictions, answers, lengths, all_classes):
    scores = dict()
    total_score = 0.0
    for (prediction, ground_truths, length) in zip(predictions, answers, lengths):
        score = 0.
        if dataset in ["trec", "triviaqa", "samsum", "lsht"]:
            prediction = prediction.lstrip('\n').split('\n')[0]
        for ground_truth in ground_truths:
            score = max(score, dataset2metric[dataset](prediction, ground_truth, all_classes=all_classes))
        total_score += score
    for key in scores.keys():
        scores[key] = round(100 * np.mean(scores[key]), 2)
    scores["total_score"] = round(100 * total_score / len(predictions), 2)
    return scores

def scorer(dataset, predictions, answers, all_classes):
    total_score = 0.
    for (prediction, ground_truths) in zip(predictions, answers):
        score = 0.
        if dataset in ["trec", "triviaqa_e", "samsum_e", "lsht"]:
            prediction = prediction.lstrip('\n').split('\n')[0]
        for ground_truth in ground_truths:
            score = max(score, dataset2metric[dataset](prediction, ground_truth, all_classes=all_classes))
        total_score += score
    return round(100 * total_score / len(predictions), 2)


pred_path = '/vepfs/wcf/G/zecheng/Retrieval_Head/quick_eval/longbench/llama-2-simpo'
benchmark_dataset_path=None
scores = dict()
all_files = os.listdir(pred_path)
print("Evaluating on:", all_files)

## extract all golden data classes
data_classes = {}
if benchmark_dataset_path is None:
    benchmark_dataset_path = "/vepfs/wcf/G/zecheng/Retrieval_Head/quick_eval/data"
all_benchmark_data_files = os.listdir(benchmark_dataset_path)
for f in all_benchmark_data_files:
    data_name = f.split('.')[0]
    content = auto_read_data(os.path.join(benchmark_dataset_path, f))
    data_classes[data_name] = content[0]["all_classes"]

## read all predicted datasets
for filename in all_files:
    if not filename.endswith("jsonl"):
        continue
    dataset_name = filename.split('.')[0]
    pred_dataset_length, all_classes = longbench_pred_length[dataset_name], data_classes[dataset_name]
    predictions, answers = [], []
    
    with open(os.path.join(pred_path, filename), "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            predictions.append(data["pred_str"])
            answers.append(data["answers"])

    if len(predictions) == 0: continue
    score = scorer(dataset_name, predictions, answers, all_classes)
    scores[dataset_name] = score

Evaluating on: ['qmsum_e.jsonl', 'qasper_e.jsonl', 'gov_report_e.jsonl', 'triviaqa_e.jsonl', '2wikimqa_e.jsonl', 'hotpotqa_e.jsonl', 'result.csv', 'trec.jsonl', 'multi_news_e.jsonl', 'passage_count.jsonl', 'samsum_e.jsonl', 'result.json', 'musique.jsonl', 'multifieldqa_en.jsonl', 'narrativeqa.jsonl', 'lcc_e.jsonl', 'passage_retrieval_en.jsonl', 'repobench-p_e.jsonl']


In [6]:
scores

{'qmsum_e': 22.15,
 'qasper_e': 22.11,
 'gov_report_e': 30.84,
 'triviaqa_e': 66.86,
 '2wikimqa_e': 20.13,
 'hotpotqa_e': 28.93,
 'trec': 63.0,
 'multi_news_e': 24.03,
 'passage_count': 1.0,
 'samsum_e': 29.97,
 'musique': 9.93,
 'multifieldqa_en': 33.39,
 'narrativeqa': 12.01,
 'lcc_e': 34.73,
 'passage_retrieval_en': 6.5,
 'repobench-p_e': 29.17}

In [19]:
json_data = scores

# 将JSON数据转换为DataFrame
df = pd.DataFrame({k: v for k, v in json_data.items()}, index=[0])

# 定义新的列顺序
columns = [
    ['Single-Document QA', 'Single-Document QA', 'Single-Document QA', 'Single-Document QA',
    'Multi-Document QA', 'Multi-Document QA', 'Multi-Document QA', 'Multi-Document QA',
    'Summarization', 'Summarization', 'Summarization', 'Summarization',
    'Few-shot Learning', 'Few-shot Learning', 'Few-shot Learning', 'Few-shot Learning',
    'Synthetic Tasks', 'Synthetic Tasks', 'Synthetic Tasks',
    'Code Completion', 'Code Completion', 'Code Completion',
    'ALL Avg'],
    ['qasper_e', 'multifieldqa_en', 'narrativeqa', 'Avg.',
    'hotpotqa_e', '2wikimqa_e', 'musique', 'Avg.',
    'gov_report_e', 'qmsum_e', 'multi_news_e', 'Avg.',
    'trec', 'triviaqa_e', 'samsum_e', 'Avg.',
    'passage_count', 'passage_retrieval_en', 'Avg.',
    'lcc_e', 'repobench-p_e', 'Avg.',
    '']
]

# 重新组织数据
new_df = pd.DataFrame(columns=columns)
for col_0, col in zip(columns[0], columns[1]):
    if col in df.columns:
        new_df[(col_0, col)] = df[col]
    elif col == 'Avg.':
        new_df[(col_0, col)] = np.nan
    else:
        new_df[(col_0, col)] = np.nan

# 计算平均值
for category in ['Single-Document QA', 'Multi-Document QA', 'Summarization', 'Few-shot Learning', 'Synthetic Tasks', 'Code Completion']:
    mask = new_df.columns.get_level_values(0) == category
    new_df.loc[:, (category, 'Avg.')] = new_df.loc[:, mask].mean(axis=1)


In [20]:
new_df

Single-Document QA                                        Multi-Document QA  \
            qasper_e multifieldqa_en narrativeqa       Avg.        hotpotqa_e   
0              22.11           33.39       12.01  22.503333             28.93   

                                Summarization          ... Few-shot Learning  \
  2wikimqa_e musique       Avg.  gov_report_e qmsum_e  ...        triviaqa_e   
0      20.13    9.93  19.663333         30.84   22.15  ...             66.86   

                      Synthetic Tasks                             \
  samsum_e       Avg.   passage_count passage_retrieval_en  Avg.   
0    29.97  53.276667             1.0                  6.5  3.75   

  Code Completion                      ALL Avg  
            lcc_e repobench-p_e   Avg.          
0           34.73         29.17  31.95     NaN  

[1 rows x 23 columns]

In [21]:
# 计算总平均值
new_df[('ALL Avg', '')] = new_df.loc[:, new_df.columns.get_level_values(1) == 'Avg.'].mean(axis=1)
new_df
# out_path = os.path.join(pred_path, "result.csv")
# # 保存为 CSV 文件
# new_df.to_csv(out_path, index=False)

Single-Document QA                                        Multi-Document QA  \
            qasper_e multifieldqa_en narrativeqa       Avg.        hotpotqa_e   
0              22.11           33.39       12.01  22.503333             28.93   

                                Summarization          ... Few-shot Learning  \
  2wikimqa_e musique       Avg.  gov_report_e qmsum_e  ...        triviaqa_e   
0      20.13    9.93  19.663333         30.84   22.15  ...             66.86   

                      Synthetic Tasks                             \
  samsum_e       Avg.   passage_count passage_retrieval_en  Avg.   
0    29.97  53.276667             1.0                  6.5  3.75   

  Code Completion                         ALL Avg  
            lcc_e repobench-p_e   Avg.             
0           34.73         29.17  31.95  26.136111  

[1 rows x 23 columns]